## Lab 04

## Zadanie 1 (Igor Sieradzki)

Funkcja <t>flow_rate</i> do modyfikacji

In [1]:
def flow_rate(weight, time, **kwargs):
    """ Funkcja wylicza ile wagi produktu przybyło/ubyło w jednostce czasu """
    return (weight * kwargs.get('units_per_kg', 1)) / (time / kwargs.get('period', 1))

weight = 0.5
time = 3

In [ ]:
flow = flow_rate(weight, time)
print("{0:.3} kg per second".format(flow))

In [ ]:
Zmodyfikuj funckje <i>flow_rate</i> tak, aby poniższe wywołania działały poprawnie.

In [2]:
flow = flow_rate(weight, time, period=60, units_per_kg=1000)
print("{} grams per minute".format(flow))

10000.0 grams per minute


In [3]:
flow = flow_rate(weight, time, period=1, units_per_kg=1)
print("{0:.3} kg per second".format(flow))

0.167 kg per second


In [4]:
flow = flow_rate(weight, time)
print("{0:.3} grams per minute".format(flow))

0.167 grams per minute


Wyjaśnić czemu poniższe rzucanie błędu jest porządane

In [5]:
try:
    flow = flow_rate(weight, time, 3600, 2.2)
except TypeError:
    print(True)

True


## Zadanie 2 (Igor Sieradzki)

Dopisz definicje dekoratora _timeit_, który wypisze na wyjscie standardowe czas wywołania udekorowanej funckji np.

```
@timeit
def foo(x):
    return x**2
    
r = foo(2)
```
wypize: <br>
    `Function foo took: 0.00001 seconds` <br>
oraz zwróci 4

In [11]:
from time import time

def timeit(func):
    """ Wypisuje czas wywołania udekorowanej funckji """
    @wraps(func)
    def get_time_between_runs(*args):
        start = time()
        func(*args)
        elapsed = time() - start
        print(f"Function {func.__name__} took {elapsed} seconds")
    return get_time_between_runs

@timeit
def squares_list(n):
    squares = []
    for i in range(n):
        squares.append(i ** 2)
    return squares

@timeit
def squares_comprehension(n):
    return [i ** 2 for i in range(n)]

@timeit
def squares_map(n):
    return map(lambda x: x**2, range(n))

n = 1000000
l = squares_list(n)
c = squares_comprehension(n)
m = squares_map(n)

Function squares_list took 0.4179267883300781 seconds
Function squares_comprehension took 0.34078001976013184 seconds
Function squares_map took 1.4543533325195312e-05 seconds


## Zadanie 3 (Igor Sieradzki)

Dopisz definicje dekoratora _derivate_ wg. instrukcji w _docstringu_

In [41]:
import sys
# sys.float_info.epsilon # epsilon maszynowy


def derivate(epsilon=1000 * sys.float_info.epsilon):
    """
    Zwraca pochodną funkcji w punkcie, wg. wzoru f'(x) = [f(x+h) - f(x)]/h, 
    gdzie h jest parametrem dekoratora, jeśli nie zostanie podany, należy przyjąć 1000 * epsilon maszynowy
    """
    @wraps(func)
    def do_derivate(func):
        def calculate(arg):
            return (func(arg + epsilon) - func(arg)) / epsilon
        return calculate
    return do_derivate


@derivate(0.01)
def f(x):
    return x*x

@derivate(0.00001)
def g(x):
    return x*x*x+3

def test(a, b, eps=1):
    return abs(round(a)-round(b)) < eps

print(test(f(100), 200.0))
print(round(f(0)) == 0.0)

print(test(g(100), 30000.0))
print(round(g(0)) == 0.0)

from random import random
for x in [random()*1000. for _ in range(20)]:
    print(f(x), 2*x, '\t', test(f(x), 2*x))
    print(g(x), 3*x**2, '\t', test(g(x), 3*x**2))

True
True
True
True
1796.805719844997 1796.7957198493862 	 True
2421356.153488159 2421356.1441518054 	 True
904.6866038494045 904.676603850824 	 True
613829.8287987709 613829.8181662955 	 True
1788.7435542768799 1788.733554281357 	 True
2399675.8222579956 2399675.7961590122 	 True
211.6350228585361 211.62502285841512 	 True
33588.86591158807 33588.862724868544 	 True
1744.2789674270898 1744.2689674280243 	 True
2281855.7024002075 2281855.6730493195 	 True
191.6676220977024 191.65762209760052 	 True
27549.485978670415 27549.48308107999 	 True
791.0308929043822 791.0208929034654 	 True
469285.549968481 469285.5397573468 	 True
256.0072206912082 255.99722069123155 	 True
49150.93660820275 49150.93275122633 	 True
557.8932032076409 557.8832032082937 	 True
233425.25884509084 233425.25131645976 	 True
1115.0514311389998 1115.041431138337 	 True
932488.0599975585 932488.044866273 	 True
563.5461265032063 563.5361265038295 	 True
238179.73211407659 238179.72440620503 	 True
670.2685991418548 

## Zadanie 4 (Igor Sieradzki)

Dopisz definicje dekoratora _accepts_ wg. instrukcji w _docstringu_

In [26]:
def accepts(*types):
    """Sprawdza czy udekorowanej funckji zostały podane odpowiednie parametry zdefiniowane 
       w argumentach dekoratora"""
    def decorator(func):
        @wraps(func)
        def check_types(*args, **kwargs):
            if not all([isinstance(arg, type) for type, arg in zip(types, args)]):
                raise TypeError()
            return func(*args, **kwargs)
        return check_types
    return decorator


@accepts(str)
def capitalize(word):
    return word[0].upper() + word[1:]

print(capitalize('ola') == 'Ola')

try:
    capitalize(2)
except TypeError:
    print(True)

@accepts(float, int)
def static_pow(base, exp):
    return base ** exp 

print(static_pow(2., 2) == 4.)
print(static_pow(2., exp=2) == 4.)
print(static_pow(base=2., exp=2) == 4.)

try:
    static_pow('x', 10)
except TypeError:
    print(True)
    
try:
    static_pow(2, 2.2)
except TypeError:
    print(True)

True
True
True
True
True
True
True


## Zadanie 5 (Igor Sieradzki)

Dopisz definicje dekoratora _returns_ wg. instrukcji w _docstringu_

In [28]:
from re import split


def returns(*types):
    """Sprawdza czy udekorowana funkcja zwraca poprawne argumenty, zdefiniowane w parametrach dekoratora"""
    def decorator(func):
        @wraps(func)
        def check_types(*args, **kwargs):
            result = func(*args, **kwargs)
            if not all([isinstance(arg, type) for type, arg in zip(types, result)]):
                raise TypeError()
            return result
        return check_types
    return decorator


@returns(str)
def str_only_identity(word):
    return word

print(str_only_identity('hello') == 'hello')

try:
    str_only_identity(10)
except TypeError:
    print(True)
    
@returns(int, int)
def split_indices(x):
    return x[0], x[1]

print(split_indices(x=[6,9]) == (6,9))

try:
    split('AB')
except TypeError:
    print(True)

True
True
True
True


## Zadanie 6 (Igor Sieradzki)
Stwórz dekorator cached służący do cachowania wywołań dowolnej funkcji, tzn. chcemy by:
* wywołanie funkcji z określonymi argumentami miało miejsce tylko raz
* funkcja mogła przyjmować dowolną liczbę nazwanych i nienazwanych argumentów
* nie musi reagować poprawnie na domyślne argumenty, tzn. wywołanie funkcji z domyślnymi argumentami a podanie dokładnie takich samych może być traktowane jako dwa różne wywołania
* na opakowanej funkcji można wywołać `.cache_reset()`, który usunie cache z pamięci
* wywołanie `.cache_status()` zwraca string z opisem w postaci: <br>
    `Function FUNCTION_NAME called X times, evaluated Y times`

In [49]:
import functools
from functools import wraps
from random import random

class Cached:
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.times_run = 0
        self.times_executed = 0
        self.cache = {}

    @staticmethod
    def _hash(name, *args, **kwargs):
        return f'{name}{str(args)}{str([f"{key}={value}" for key, value in sorted(kwargs.items())])}'

    def cache_reset(self):
        self.cache = {}

    def cache_status(self):
        return f'Function {self.func.__name__} called {self.times_run} times, evaluated {self.times_executed} times'

    def __call__(self, *args, **kwargs):
        exec_hash = self._hash(self.func.__name__, *args, **kwargs)
        self.times_run += 1
        if exec_hash in self.cache:
            return self.cache[exec_hash]
        self.times_executed += 1
        return self.cache.setdefault(exec_hash, self.func(*args, **kwargs))

def cached(func):
    def hash(name, *args, **kwargs):
        return f'{name}{"".join(map(str, args))}{"".join([f"{key}={value}" for key, value in sorted(kwargs.items())])}'

    @wraps(func)
    def decorator(*args, **kwargs):
        exec_hash = hash(func.__name__, *args, **kwargs)
        decorator.times_run += 1
        if exec_hash in decorator.cache:
            return decorator.cache[exec_hash]
        decorator.times_executed += 1
        return decorator.cache.setdefault(exec_hash, func(*args, **kwargs))

    def cache_reset():
        decorator.cache = {}

    def cache_status():
        return f'Function {func.__name__} called {decorator.times_run} times, evaluated {decorator.times_executed} times'

    decorator.times_run = 0
    decorator.times_executed = 0
    decorator.cache_status = cache_status
    decorator.cache_reset = cache_reset
    cache_reset()

    return decorator

@Cached
def foo(x, y=1, z=4):
    return random()

@Cached
def abcd():
    return 1
        
print(foo(3) == foo(3))
print(foo(4) == foo(4))
print(foo(3, z=-1, y=3) == foo(3, y=3, z=-1))
print(foo(3) != foo(x=3))
a = foo(3)
foo.cache_reset()
print(a != foo(3))
print(foo.cache_status() == 'Function foo called 10 times, evaluated 5 times')
abcd()
abcd()
abcd()
print(abcd.cache_status())

True
True
False
True
True
False
Function abcd called 3 times, evaluated 1 times


## Zadanie 7 (Krzysztof Hajto)

Napisz dekorator który będzie robić n-krotne złożenie funkcji, gdzie n jest parametrem dekoratora

In [40]:
from functools import reduce


def zlozenie(n):
    def decorator_with_arg(func):
        @wraps(func)
        def decorator(arg):
            result = func(arg)
            for i in range(n - 1):
                result = func(result)
            return result
        return decorator
    return decorator_with_arg

@zlozenie(3)
def f1(x):
    return x+1

@zlozenie(2)
def f2(x):
    return x*x

@zlozenie(5)
def f3(word):
    return "".join(chr(ord(l)+1) for l in word)

print(f1(2)==5)
print(f2(3)==81)
print(f3("alamakota")=="fqfrfptyf")

True
True
True


## Zadanie 8 (Krzysztof Hajto)

Python nie ma wbudowanej instrukcji switch. Ale posiada anonimowe funkcje oraz słowniki. Zaimplementuj poniższy switch w postaci słownika funkcji.
`
int my_function(x, y) {


    switch(x) {
        case 1: return y*y;
        case 2: return x+y;
        case 3: return x*y;
        case 4: return 0;
    }
}
`

PS. Nigdy nie róbcie tego w faktycznym kodzie :)

In [39]:
def my_function(x, y):
    return {
        1: lambda: y * y,
        2: lambda: x + y,
        3: lambda: x * y,
        4: lambda: 0
    }[x]()

print(my_function(1,3)==9)
print(my_function(2,4)==6)
print(my_function(3,1)==3)
print(my_function(4,9)==0)

True
True
True
True
